In [17]:
import sys
import pandas as pd
from datetime import datetime,timedelta
sys.path.append('../scripts')
from sql_engine import connect
import batch_failure as bf
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
fn_engine = connect('fn_mysql')
sql_query = r'''
                select
                ID,
                `PO Number`,
                `Lot Number`,
                `Vendor Code`,
                Vendor,
                Division,
                `Inspection Date`,
                Inspector,
                `Item Number`,
                Results,
                `Reject Code`,
                `Reject Description`,
                `Comments`,
                `Path`
            from
                inspection_data_all
            where 1 = 1
            and `Inspection Date` >= '2023-07-01'
            and Results = 'R'
            and `Reject Description` like '%%100\%%%%'
            order by `Inspection Date`   
            '''

In [61]:
df_inspection = pd.read_sql(sql_query,fn_engine)

In [62]:
df_QIM_duplicate = (
    df_inspection
    .query('Path.str.startswith("QIM",na=False)')
    .sort_values(['ID','Inspection Date'],ascending=[False,False])
    .drop_duplicates(subset=['ID'],keep='last')
)

In [63]:
df_offline_duplicate = (
    df_inspection
    .query('~Path.str.startswith("QIM",na=False)')
    .sort_values(['ID','Inspection Date'],ascending=[False,False])
    .drop_duplicates(subset=['Lot Number','Vendor','Item Number','Inspector','Inspection Date'],keep='last')
)

In [66]:
df_duplicate = pd.concat([df_QIM_duplicate,df_offline_duplicate],ignore_index=True)